In [ ]:
## Model Details

# DeBERTa-v3-large


In [1]:
## Install Libraries

!pip install transformers
!pip install tqdm
!pip install torch
!pip install pandas
!pip install sklearn
!pip install numpy
!pip install nltk
!pip install kaggle
!pip install tokenizers
!pip install sentencepiece
!pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.1 MB/s 
     |████████████████████████████████| 101 kB 11.6 MB/s 
     |████████████████████████████████| 596 kB 57.6 MB/s 
     |████████████████████████████████| 6.6 MB 40.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https:

In [2]:
!unzip '/content/Essay/train-essay.zip' -d '/content/Essay'

Archive:  /content/Essay/train-essay.zip
  inflating: /content/Essay/train-essay/00066EA9880D.txt  
  inflating: /content/Essay/train-essay/000E6DE9E817.txt  
  inflating: /content/Essay/train-essay/0016926B079C.txt  
  inflating: /content/Essay/train-essay/00203C45FC55.txt  
  inflating: /content/Essay/train-essay/0029F4D19C3F.txt  
  inflating: /content/Essay/train-essay/0045BE2791A2.txt  
  inflating: /content/Essay/train-essay/004AC288D833.txt  
  inflating: /content/Essay/train-essay/0056F3D261D5.txt  
  inflating: /content/Essay/train-essay/007ACE74B050.txt  
  inflating: /content/Essay/train-essay/007E0CBA8852.txt  
  inflating: /content/Essay/train-essay/008C457C8E0B.txt  
  inflating: /content/Essay/train-essay/00944C693682.txt  
  inflating: /content/Essay/train-essay/00B144412785.txt  
  inflating: /content/Essay/train-essay/00BD97EA4041.txt  
  inflating: /content/Essay/train-essay/00C6E82FE5BA.txt  
  inflating: /content/Essay/train-essay/00D304153840.txt  
  inflating: /c

In [ ]:
#! gdown --id 1cOB8QzCbzNJ-ZxUzfWDID7zuk2EOSneL

In [3]:
## Import

import tokenizers

import transformers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers.models.deberta_v2 import DebertaV2TokenizerFast
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

from tqdm.auto import tqdm

import sentencepiece
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd

from sklearn import model_selection
from sklearn import metrics

import string
import os
import inspect

import nltk
from nltk.corpus import words
from nltk.corpus import brown
from nltk.corpus import wordnet
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
nltk.download('words')
nltk.download('punkt')
nltk.download('brown')
nltk.download('wordnet')
nltk.download('omw-1.4')

ps = PorterStemmer()
from spellchecker import SpellChecker
spell = SpellChecker()


#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [4]:
## Creating Word Set for Not In Index

word_set = {'sample'}

def add_words(word_set, words):
    index = 0
    for word in words:
        word = word.lower()
        if index % 100000 == 0:
            print(f'Processed {index} words')
        index += 1    
    
        if not word in word_set:
            word_set.add(word)
        
    print(len(word_set))
    
add_words(word_set, brown.words())
add_words(word_set, words.words())
add_words(word_set, wordnet.words())

Processed 0 words
Processed 100000 words
Processed 200000 words
Processed 300000 words
Processed 400000 words
Processed 500000 words
Processed 600000 words
Processed 700000 words
Processed 800000 words
Processed 900000 words
Processed 1000000 words
Processed 1100000 words
49815
Processed 0 words
Processed 100000 words
Processed 200000 words
261552
Processed 0 words
Processed 100000 words
346423


In [5]:
VERSION = 'v21Final'

MAX_LEN_DISCOURSE_TEXT = 1024
TRAIN_BATCH_SIZE  = 4
VALID_BATCH_SIZE = 2
EPOCHS = 5
DROP_OUT = 0.10
TEST_SIZE = 0.1
LEARNING_RATE = 3e-6


BASE_MODEL = 'microsoft/deberta-v3-base'
MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/Model/model' + VERSION + '.bin'

TRAINING_FILE =  '/content/Data/train_berkeley.csv'
TEST_FILE = '/content/Data/test_berkeley.csv'
ESSAY_FOLDER = '/content/Essay/train-essay'

TOKENIZER = DebertaV2TokenizerFast.from_pretrained(BASE_MODEL)


CLASS_MAPPING = {
    'Adequate': 1,
    'Effective': 0,
    'Ineffective' : 2
}

DISCOURSE_TYPE_MAPPING = {
    'Lead': 0,
    'Position': 1,
    'Claim' : 2,
    'Evidence' : 3,
    'Counterclaim' : 4,
    'Rebuttal' : 5,
    'Concluding Statement' : 6
}

device = torch.device("cuda")
torch.cuda.empty_cache()
print(f'Device: {device}')

GPT_MODEL_ID = 'gpt2'  # can also change to gpt-2 large if size is not an issue
GPT_MODEL = GPT2LMHeadModel.from_pretrained(GPT_MODEL_ID).to(device)
GPT_TOKENIZER = GPT2TokenizerFast.from_pretrained(GPT_MODEL_ID)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Device: cuda


In [6]:
# Dataset Class

class BERTDataset:
    def __init__(self, discourse_texts, discourse_types, essay_ids, targets, essay_folder, max_len_discourse_text, word_set, discourse_types_text):
        self.discourse_texts = discourse_texts
        self.discourse_types = discourse_types
        self.discourse_types_text = discourse_types_text
        self.targets = targets
        self.tokenizer = TOKENIZER
        self.essay_ids = essay_ids
        self.max_len_discourse_text = max_len_discourse_text        
        self.essay_folder = essay_folder
        self.word_set = word_set

    def __len__(self):
        return len(self.discourse_texts)
    
    def __getitem__(self, item):
        discourse_text = str(self.discourse_texts[item])
        discourse_type = self.discourse_types[item]        
        discourse_type_text = self.discourse_types_text[item]

        # Counting Spelling errors in Text
        text = discourse_text
        text = text.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
        text = text.lower()
        tokens = word_tokenize(text)
    
        count = 0
        for token in tokens:            
            if not token in word_set and not str.isdigit(token):
                token_stem = ps.stem(token)
                if not token_stem in word_set:
                    count += 1
        
        nii_count_discourse_text = (count - 0.85) / 1.70
        
        gpt_encodings = GPT_TOKENIZER(discourse_text, return_tensors='pt')
        gpt_input_ids = gpt_encodings.input_ids.to(device)
        with torch.no_grad():
            gpt_outputs = GPT_MODEL(gpt_input_ids, labels=gpt_input_ids)
            perplexity =  float(torch.exp(gpt_outputs[0]))
            perplexity = (perplexity - 230) / 1578
        
        
        essay_id = self.essay_ids[item]
        essay_path = os.path.join(self.essay_folder, f"{essay_id}.txt")
        essay = open(essay_path, 'r').read()
        
        input_text = discourse_type_text + " " + self.tokenizer.sep_token + discourse_text + " " + self.tokenizer.sep_token + " " + essay
        
        inputs = self.tokenizer.encode_plus(
            input_text,
            add_special_tokens=True,            
            max_length=self.max_len_discourse_text,
            truncation=True,
            padding='max_length',
            return_tensors='pt',
            return_attention_mask = True
        )

        ids = inputs.input_ids.flatten()
        mask = inputs.attention_mask.flatten() 


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'discourse_type': torch.tensor(discourse_type, dtype=torch.long),
            'target': torch.tensor(self.targets[item], dtype=torch.long),
            'nii_count_discourse_text': torch.tensor(nii_count_discourse_text, dtype=torch.float),
            'perplexity': torch.tensor(perplexity, dtype=torch.float)
        }

In [ ]:
#inspect.getfullargspec(TOKENIZER.encode_plus)

In [7]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()  
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)   
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings


class BERTBaseUncased(nn.Module):
    def __init__(self):
        super(BERTBaseUncased, self).__init__()
        self.config = AutoConfig.from_pretrained(BASE_MODEL, output_hidden_states=True)
        self.model = AutoModel.from_pretrained(BASE_MODEL, config=self.config) 
        self.mpool = MeanPooling()
        self.dropout = nn.Dropout(DROP_OUT)
        self.hidden = nn.Linear(777, 128)
        self.out = nn.Linear(128, 3)
        

    def forward(self, ids, mask, discourse_type, nii_count_discourse_text, perplexity):
        
        discourse_text_output = self.model(input_ids=ids, attention_mask=mask)
        discourse_text_hidden_states = self.mpool(discourse_text_output.last_hidden_state, mask)
        model_output_discourse_text = self.dropout(discourse_text_hidden_states)
        
        discourse_type_output = F.one_hot(discourse_type, num_classes=len(DISCOURSE_TYPE_MAPPING))        
        nii_count_discourse_text  = nii_count_discourse_text.view(-1,1)        
        perplexity = perplexity.view(-1,1)
        
        merged_output = torch.cat((model_output_discourse_text,                                   
                                   discourse_type_output,
                                   nii_count_discourse_text,
                                   perplexity), dim=1)      
        
        hidden = self.hidden(merged_output)
        output = self.out(hidden)
        
        return output


In [8]:
device = torch.device("cuda")
torch.cuda.empty_cache()
print(f'Device: {device}')

Device: cuda


In [14]:
df_train = pd.read_csv(TRAINING_FILE)
df_train['label'] = df_train.discourse_effectiveness
df_train.label = df_train.label.map(CLASS_MAPPING)
df_train['discourse_type_int'] = df_train.discourse_type
df_train.discourse_type_int = df_train.discourse_type_int.map(DISCOURSE_TYPE_MAPPING)
print(f'Total samples in Train: {len(df_train)}')


df_valid = pd.read_csv(TEST_FILE)
df_valid['label'] = df_valid.discourse_effectiveness
df_valid.label = df_valid.label.map(CLASS_MAPPING)
df_valid['discourse_type_int'] = df_valid.discourse_type
df_valid.discourse_type_int = df_valid.discourse_type_int.map(DISCOURSE_TYPE_MAPPING)
print(f'Total samples in Validation: {len(df_valid)}')


train_dataset = BERTDataset(
    discourse_texts=df_train.discourse_text.values,
    essay_ids = df_train.essay_id.values,
    targets=df_train.label.values,
    discourse_types = df_train.discourse_type_int,
    essay_folder = ESSAY_FOLDER,
    max_len_discourse_text = MAX_LEN_DISCOURSE_TEXT,
    word_set = word_set,
    discourse_types_text = df_train.discourse_type.values
)


train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True
)

valid_dataset = BERTDataset(
    discourse_texts=df_valid.discourse_text.values,
    essay_ids = df_valid.essay_id.values,
    targets=df_valid.label.values,
    discourse_types = df_valid.discourse_type_int,
    essay_folder = ESSAY_FOLDER,
    max_len_discourse_text = MAX_LEN_DISCOURSE_TEXT,
    word_set = word_set,
    discourse_types_text = df_valid.discourse_type.values
)

valid_data_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=VALID_BATCH_SIZE
)


print(f'Total Batches in Train: {len(train_data_loader)}')
print(f'Total Batches in Valid: {len(valid_data_loader)}')

Total samples in Train: 33297
Total samples in Validation: 3468
Total Batches in Train: 8325
Total Batches in Valid: 1734


In [ ]:
"""
def loss_fn(output, target):
    
    gamma = torch.tensor(2.0).to(device, dtype=torch.float)
    max_prob = torch.tensor(1.0).to(device, dtype=torch.float)
    
    output_softmax = torch.softmax(output, dim=1)
    output_softmax_nlog = -1 * torch.log(output_softmax)
    
    target_onehot = F.one_hot(target, num_classes=3)
    cross_entropy = torch.multiply(target_onehot, output_softmax_nlog)
    
    weight = torch.multiply(target_onehot, torch.pow(torch.subtract(max_prob, output_softmax), gamma))    
    
    focal_loss = torch.multiply(weight, cross_entropy)
    loss = (focal_loss).sum(dim=1).mean()
    return loss

"""

def loss_fn(output, target):
    return nn.CrossEntropyLoss()(output, target)

In [ ]:
model = BERTBaseUncased()
model.to(device)
#model.load_state_dict(torch.load(MODEL_PATH))

param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.005,
    },
    {
        "params": [
            p for n, p in param_optimizer if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.0,
    },
]

num_train_steps = int(len(df_train) / TRAIN_BATCH_SIZE * EPOCHS)
optimizer = AdamW(optimizer_parameters, lr=LEARNING_RATE)


scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=500,
    num_training_steps=num_train_steps
)



best_loss = float('inf')
best_accuracy = 0.0
total_valid_loss = 0.0
total_train_loss = 0.0
print(f'Model Training Started')

for epoch in range(EPOCHS):
        
    total_train_loss = 0.0
    # Train Function
    
    for batch_index, data in tqdm(enumerate(train_data_loader), total=len(train_data_loader)):
        ids = data['ids']
        mask = data['mask']
        target = data['target']        
        discourse_type = data['discourse_type']
        nii_count_discourse_text= data['nii_count_discourse_text']
        perplexity = data['perplexity']

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        target = target.to(device, dtype=torch.long)
        discourse_type = discourse_type.to(device, dtype=torch.long)
        nii_count_discourse_text = nii_count_discourse_text.to(device, dtype=torch.float)
        perplexity = perplexity.to(device, dtype=torch.float)

        optimizer.zero_grad()
        output = model(
            ids=ids,
            mask=mask,
            discourse_type=discourse_type,
            nii_count_discourse_text=nii_count_discourse_text,
            perplexity=perplexity
        )
        
        loss = loss_fn(output, target)
        
        with torch.no_grad():
            total_train_loss += loss.item()
        
        loss.backward()
        

        optimizer.step()
        scheduler.step()
    
    
    total_valid_loss = 0.0    
    total_correct = 0
    kaggle_loss = 0.0
    with torch.no_grad():
        model.eval()
        for batch_index, data in tqdm(enumerate(valid_data_loader), total=len(valid_data_loader)):
            ids = data['ids']
            mask = data['mask']
            target = data['target']        
            discourse_type = data['discourse_type']
            nii_count_discourse_text= data['nii_count_discourse_text']
            perplexity = data['perplexity']

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            target = target.to(device, dtype=torch.long)
            discourse_type = discourse_type.to(device, dtype=torch.long)
            nii_count_discourse_text = nii_count_discourse_text.to(device, dtype=torch.float)
            perplexity = perplexity.to(device, dtype=torch.float)

            output = model(
                ids=ids,
                mask=mask,
                discourse_type=discourse_type,
                nii_count_discourse_text=nii_count_discourse_text,
                perplexity=perplexity         
            )

            output = torch.softmax(output, dim=1)
            output_sum = torch.sum(output, dim=1)
            for batch_index in range(len(target)):
                kaggle_loss += -1 * torch.log(output[batch_index][target[batch_index]] / output_sum[batch_index])


            validloss = loss_fn(output, target)
            total_valid_loss += validloss.item()
            
            labels_prediction_conf, labels_prediction_class = torch.max(output, 1)
            total_correct += torch.sum(labels_prediction_class == target).item()
            validation_accuracy = total_correct / len(valid_dataset)
        
        
        total_valid_loss = total_valid_loss / len(valid_data_loader)
        total_train_loss = total_train_loss / len(train_data_loader)        
        print(f'Epoch: {epoch + 1} :: Training Loss: {total_train_loss:.4f}, Validation Loss: {total_valid_loss:.4f}, Validation Accuracy: {validation_accuracy:.4f}')
        print(f'Kaggle Loss: {kaggle_loss / len(valid_dataset):.4f}')        
        
        
        if validation_accuracy > best_accuracy:
            torch.save(model.state_dict(), MODEL_PATH)
            best_accuracy = validation_accuracy
            
        

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model Training Started


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/8325 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


...

Epoch: 3 :: Training Loss: 0.5043, Validation Loss: 0.8558, Validation Accuracy: 0.7111
Kaggle Loss: 0.6609


  0%|          | 0/8325 [00:00<?, ?it/s]

  0%|          | 0/1734 [00:00<?, ?it/s]

Epoch: 4 :: Training Loss: 0.4179, Validation Loss: 0.8527, Validation Accuracy: 0.7033
Kaggle Loss: 0.7030


  0%|          | 0/8325 [00:00<?, ?it/s]